# Converting SWC files to arrays

We can't feed the SWC files directly into our neural network. First, we must connect the nodes, then somehow load them into numpy.

In the interest of time, I will use already-existing Vaa3D plugins to first render 3D TIFF images, which can then be loaded with numpy.

First, I want a list of file names.

In [1]:
from swc_io import get_fnames_and_abspath_from_dir, swc_to_TIFF

MEAN_CENTERED_CUBES_DIR = "../data/06_centered_cubes"
CUBES_DIRECTORY = 'cubes_directory.txt'




#with open(CUBES_DIRECTORY, 'w+') as cubes_directory:
#    for cube_swc_path in abs_paths:
#      cubes_directory.write("%s\n" % cube_swc_path)

In [2]:
import tkinter as tk
from tkinter import filedialog

root = tk.Tk()
root.withdraw()

vaa3d_path = filedialog.askopenfilename(title='Select compiled Vaa3d binary')

In [ ]:
from threader import multithreading

#fnames, abs_paths = get_fnames_and_abspath_from_dir(BRANCHES_DIR)
fnames, abs_paths = get_fnames_and_abspath_from_dir(MEAN_CENTERED_CUBES_DIR)
assert(len(abs_paths)>0)

vaa3d_repeated = [vaa3d_path for _ in range(len(abs_paths))]

multithreading(swc_to_TIFF, zip(fnames, abs_paths, vaa3d_repeated), 16)

In [ ]:
from pyspark import SparkContext

sc = SparkContext(appName='SWC2npy')

def combine_abstract_text(text1, text2):
    """markovify works best when each corpus is a single huge string. theefore,
    reduce by key here"""
    print(text1[:20], text2[:20])
    return text1+text2

def text_to_model(tup):
    '''given an abstract, train a markov model

    the 1 will be used for weights, later'''
    _, text = tup
    try:
        # retain_original set to False to save lots of RAM
        text_model = markovify.Text(text, state_size=STATE_SIZE, \
                                    retain_original=False)

        # class is not serializable, so extract json first
        # this makes a Text type object, so we coerce to str
        model_json = str(text_model.to_json())
        # TODO: change key for category
        return _, model_json
    except:
        # TODO FIXME: many articles being lost due to illegal characters. see issue tracker.
        print("model skipped in text_to_model:", text[:50])
        pass


def combine_models(model_1, model_2):
    """this should come in with no key"""
    print("mod1", model_1[:10])
    print("mod2", model_2[:10])
    jsons = []
    for tup in unzipped:
        tup = tup[0]  # unnest 1 level
        if tup is None:
            continue  # FIXME I don't know how these Nonetypes keep sneaking in
        try:
            _name, json = tup
            jsons.append(json)
        except ValueError:
            print("combining failed", tup)

    # reconstitute classes from json
    reconstituted_models = [markovify.Text.from_json(json_i) for json_i in jsons]

    # hella redundant but combine() method only smashes 2 models at a time
    combined_model = reconstituted_models.pop()
    #weights = [1., 1.]
    for model in reconstituted_models:
        combined_model = markovify.combine([model, combined_model]) #, weights)
        #weights[-1] += 1
    combined_json = str(combined_model.to_json())
    # TODO: change key for category
    return "_", combined_json


def model_to_json(model):
    try:
        model_name, model_json = model
    except TypeError:  # TODO FIXME somehow STILL Nonetypes leaking through
        print("model was a Nonetype, not saving squat")
        return None
    if SAVE_MODELS:
        fname = open('./models/{}_model.json'.format(model_name), 'w')
        fname.write(model_json)
        fname.close
        print("wrote json to disk for model {}".format(model_name))


abstracts = sc.textFile(CUBES_DIRECTORY)
abstracts = abstracts.map(split_line)
abstracts = abstracts.filter(lambda tup: tup[1] is not None)
abstracts = abstracts.filter(lambda tup: len(tup[1]) >= 12)
#print(abstracts.take(1))
abstracts = abstracts.reduceByKey(lambda text1, text2: text1+text2)
abstracts.persist()  # do not lose RDD after next line
print("# of words in each key/corpus: ", abstracts.map(lambda tup: len(tup[1])).collect())
models = abstracts.map(text_to_model)
#print(models.take(1))
#combined_models = models.reduceByKey(combine_models)
#print(combined_models.take(1))
# I like this function better, except is isnt' working anymore and I can't figure out why
#models.map(model_to_json)  
# rdd.saveAsTextFile saves as tuples, which sucks
if SAVE_MODELS:
    model_dir = "models/"
    from shutil import rmtree
    rmtree(model_dir)
    models.saveAsTextFile(model_dir)

